In [10]:
import numpy as np
from data_loader import load_dataset, load_challenge, save_challenge
from torch import nn
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
from eval_script import compute_revenue

In [11]:
class BirdCrossEntropyLoss(nn.Module):
    def __init__(self, device):
        super(BirdCrossEntropyLoss, self).__init__()
        self.rev_matrix = torch.tensor(np.array(
            [[0.05, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2],
             [-0.25,  1., -0.3, -0.1, -0.1, -0.1, -0.1],
                [-0.02, -0.1,  1., -0.1, -0.1, -0.1, -0.1],
                [-0.25, -0.1, -0.3,  1., -0.1, -0.1, -0.1],
                [-0.25, -0.1, -0.3, -0.1,  1., -0.1, -0.1],
                [-0.25, -0.1, -0.3, -0.1, -0.1,  1., -0.1],
                [-0.25, -0.1, -0.3, -0.1, -0.1, -0.1,  1.]])).to(device)
        self.cet = nn.CrossEntropyLoss(reduction='none').to(device)


    def forward(self, input, target):
        ce_loss = self.cet(input, target)

        input_labels = torch.argmax(input, dim=1)
        target_labels = torch.argmax(target, dim=1)
        wanted = self.rev_matrix[target_labels, target_labels]
        actual = self.rev_matrix[target_labels, input_labels]
        cost = wanted - actual
        custom_loss = cost * ce_loss

        return custom_loss.mean()


In [12]:
class BirdDataset(Dataset):
    def __init__(self, birdset, is_train, device, offset, scaler=None):
        self.device = device
        self.offset = offset
        self.data = np.empty((0, 548), dtype=np.float32)
        self.labels = np.empty((0), dtype=int)
        for bird in birdset:
            if is_train:
                self.data = np.concatenate(
                    (self.data, np.concatenate(birdset[bird]['train_features'])))
                self.labels = np.concatenate(
                    (self.labels, np.concatenate(birdset[bird]['train_labels'])))
            else:
                self.data = np.concatenate(
                    (self.data, np.concatenate(birdset[bird]['test_features'])))
                self.labels = np.concatenate(
                    (self.labels, np.concatenate(birdset[bird]['test_labels'])))

        self.data = np.concatenate((np.zeros((offset, 548)), self.data, np.zeros((offset, 548))))

        hot_matrix = np.eye(7)
        self.labels = hot_matrix[self.labels]

        if scaler == None:
            scaler = StandardScaler()
            scaler.fit(self.data)
        self.scaler = scaler

        self.data = torch.tensor(
            self.scaler.transform(self.data), dtype=torch.float32).to(self.device)
        self.labels = torch.tensor(self.labels).to(self.device)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        x = self.data[index:index+self.offset*2+1].unsqueeze(0)
        y = self.labels[index]
        return x, y


In [13]:
def train(dataloader, model, loss_fn, optimizer, progress_steps):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if progress_steps != None and batch % progress_steps == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn, show_progress):
    num_batches = len(dataloader)
    test_loss = 0
    money = 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            money += compute_revenue(torch.argmax(pred, dim=1).cpu(), torch.argmax(y, dim=1).cpu())

    test_loss /= num_batches
    if show_progress:
        print(f"Avg loss: {test_loss:>8f}, Money saved: {money:.2f}$\n")
    return test_loss

def eval(model_fn, train_dataloader, test_dataloader, device, max_epochs):
    model = model_fn(None).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    loss_fn = BirdCrossEntropyLoss(device)
    lowest_test_loss = np.infty
    stop_criterion = 0
    for t in range(max_epochs):
        if t % 10 == 0:
            print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer, None)
        test_loss = test(test_dataloader, model, loss_fn, True)
        if test_loss > lowest_test_loss:
            stop_criterion += 1
        else:
            lowest_test_loss = test_loss
            stop_criterion = 0
        if stop_criterion >= 10:
            break
    print("Estimated performance:")
    test(test_dataloader, model, loss_fn, True)
    return model


In [14]:
offset = 5

model_fn = lambda _: nn.Sequential(
        nn.Conv2d(1, 16, kernel_size=3, stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(16, 32, kernel_size=3, stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(32 * 135 * 1, 64),
        nn.ReLU(),
        nn.Linear(64, 7)
    )

In [6]:
dataset = load_dataset()

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
train_set = BirdDataset(dataset, True, device, offset)
scaler = train_set.scaler
test_set = BirdDataset(dataset, False, device, offset, scaler)
train_loader = DataLoader(train_set, batch_size=32)
test_loader = DataLoader(test_set, batch_size=32)

In [16]:
model = eval(model_fn, train_loader, test_loader, device, 500)

Epoch 1
-------------------------------
Avg loss: 1.635208, Money saved: -177.88$

Avg loss: 1.552566, Money saved: -358.23$

Avg loss: 1.649484, Money saved: -643.16$

Avg loss: 1.511505, Money saved: -298.81$

Avg loss: 1.437673, Money saved: -276.61$

Avg loss: 1.353309, Money saved: -313.12$

Avg loss: 1.535532, Money saved: -344.91$

Avg loss: 1.485006, Money saved: -483.73$

Avg loss: 1.393464, Money saved: -348.46$

Avg loss: 1.405801, Money saved: -643.77$

Epoch 11
-------------------------------
Avg loss: 1.283331, Money saved: -338.22$

Avg loss: 1.386291, Money saved: -509.30$

Avg loss: 1.432459, Money saved: -485.99$

Avg loss: 1.312654, Money saved: -666.28$

Avg loss: 1.232489, Money saved: -442.28$

Avg loss: 1.310583, Money saved: -1168.53$

Avg loss: 1.251517, Money saved: -385.38$

Avg loss: 1.209856, Money saved: -885.43$

Avg loss: 1.073426, Money saved: -743.62$

Avg loss: 1.155390, Money saved: -648.67$

Epoch 21
-------------------------------
Avg loss: 1.01198

In [17]:
torch.save(model.state_dict(), 'cnn')

In [18]:
challenge = load_challenge()
print(challenge.shape)

(16, 3000, 548)


In [19]:
model = model_fn(None)
model.load_state_dict(torch.load('cnn'))
model.eval()

model = model.to(device)
results = torch.Tensor(np.empty((challenge.shape[0], challenge.shape[1]))).to(device)
for i in range(challenge.shape[0]):
    challenge_subset = challenge[i]
    challenge_subset = scaler.transform(challenge_subset)
    challenge_subset = torch.Tensor(challenge_subset).unsqueeze(0).to(device)
    preds = model(challenge_subset).squeeze()
    pred_labels = torch.argmax(preds, dim=1)
    results[i, :] = pred_labels
results = results.int().cpu().numpy()
print(results, results.shape)
save_challenge('cnn', results)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x100980 and 4320x64)